# 责任链
亦称：职责链模式、命令链、CoR、Chain of Command、Chain of Responsibility  
责任链是一种行为设计模式，允许你将请求沿着处理者链进行发送。收到请求后，每个处理者均可对请求进行处理，或将其传递给链上的下个处理者。


## Keyword
编写一级处理器抽象类，分别具有设置链条下一级和功能实现两个抽象函数，然后编写二级处理器抽象类，实现具体的链条设置和链条传递。最后编写处理器实类，实现更特殊性的功能。

## 最佳抽象

In [1]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional


class Handler(ABC):
    """
    The Handler interface declares a method for building the chain of handlers.
    It also declares a method for executing a request.
    """

    @abstractmethod
    def set_next(self, handler: Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass


class AbstractHandler(Handler):
    """
    The default chaining behavior can be implemented inside a base handler
    class.
    """

    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        # Returning a handler from here will let us link handlers in a
        # convenient way like this:
        # monkey.set_next(squirrel).set_next(dog)
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return None


"""
All Concrete Handlers either handle a request or pass it to the next handler in
the chain.
"""


class MonkeyHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Banana":
            return f"Monkey: I'll eat the {request}"
        else:
            return super().handle(request)


class SquirrelHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "Nut":
            return f"Squirrel: I'll eat the {request}"
        else:
            return super().handle(request)


class DogHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request == "MeatBall":
            return f"Dog: I'll eat the {request}"
        else:
            return super().handle(request)


def client_code(handler: Handler) -> None:
    """
    The client code is usually suited to work with a single handler. In most
    cases, it is not even aware that the handler is part of a chain.
    """

    for food in ["Nut", "Banana", "Cup of coffee"]:
        print(f"\nClient: Who wants a {food}?")
        result = handler.handle(food)
        if result:
            print(f"  {result}", end="")
        else:
            print(f"  {food} was left untouched.", end="")


if __name__ == "__main__":
    monkey = MonkeyHandler()
    squirrel = SquirrelHandler()
    dog = DogHandler()

    monkey.set_next(squirrel).set_next(dog)

    # The client should be able to send a request to any handler, not just the
    # first one in the chain.
    print("Chain: Monkey > Squirrel > Dog")
    client_code(monkey)
    print("\n")

    print("Subchain: Squirrel > Dog")
    client_code(squirrel)

Chain: Monkey > Squirrel > Dog

Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Monkey: I'll eat the Banana
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.

Subchain: Squirrel > Dog

Client: Who wants a Nut?
  Squirrel: I'll eat the Nut
Client: Who wants a Banana?
  Banana was left untouched.
Client: Who wants a Cup of coffee?
  Cup of coffee was left untouched.

## 最简实践
我们编写一个RAG的责任链，答案生成优先级依次为文档，搜索和大模型直接生成。

In [9]:
from __future__ import annotations
from abc import ABC, abstractmethod
from typing import Any, Optional

class Handler(ABC):

    @abstractmethod
    def set_next(self,handler:Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self,request) -> Optional[str]:
        pass

class AbstractHandler(Handler):

    _next_handler: Handler = None
    

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        # Returning a handler from here will let us link handlers in a
        # convenient way like this:
        # monkey.set_next(squirrel).set_next(dog)
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return None

class Doc_answer(AbstractHandler):

    def handle(self,request: Any) -> str:
        if request == "知识库问答":
            return f'知识库检索答案如下'
        else :
            return super().handle(request) 


class Search_answer(AbstractHandler):

    def handle(self,request: Any) -> str:
        if request == "搜索问答":
            return f'谷歌搜索答案如下'
        else :
            return super().handle(request) 
class LLM_answer(AbstractHandler):

    def handle(self,request: Any) -> str:
        if request == "大模型生成问答":
            return f'大模型生成答案如下'
        else :
            return super().handle(request) 
        

def client(handler: Handler) -> None:
    """
    The client code is usually suited to work with a single handler. In most
    cases, it is not even aware that the handler is part of a chain.
    """

    for ask in ["知识库问答", "搜索问答", "大模型生成问答","人工生成问答"]:
        print(f"\nClient: 我们请求使用{ask}?")
        result = handler.handle(ask)
        if result:
            print(f"  {result}", end="")
        else:
            print(f"  {ask} 无法使用.", end="")

if __name__ == "__main__" :
    doc = Doc_answer()
    search = Search_answer()
    LLM = LLM_answer()

    doc.set_next(search).set_next(LLM)

    print("Chain: 知识库问答 > 搜索问答 > 大模型生成问答")
    client(doc)

Chain: 知识库问答 > 搜索问答 > 大模型生成问答

Client: 我们请求使用知识库问答?
  知识库检索答案如下
Client: 我们请求使用搜索问答?
  谷歌搜索答案如下
Client: 我们请求使用大模型生成问答?
  大模型生成答案如下
Client: 我们请求使用人工生成问答?
  人工生成问答 无法使用.